#### Imports and setup

In [1]:
import warnings
warnings.simplefilter("ignore")

import math
import numpy as np
import scipy
import sklearn.datasets
import sklearn.metrics
import sklearn.model_selection
import sklearn.preprocessing
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
from tqdm.auto import tqdm
import functools

In [2]:
from tabdl import layers

In [3]:
device_id = 6
device = torch.device(f"cuda:{device_id}" if torch.cuda.is_available() else "cpu")

In [4]:
batch_size = 256
num_workers = 0

#### Training utils

In [5]:
def train_step(model, optimizer, dataloader, loss_fn, device, task_type):
    model.train()
    losses = []
    for batch in dataloader:
        if len(batch) == 3:
            x_cont, x_cat, y = batch
            x_cont, x_cat, y = x_cont.to(device), x_cat.to(device), y.to(device)
        if len(batch) == 2:
            x_cont, y = batch
            x_cont, y = x_cont.to(device), y.to(device)
            x_cat = None
        optimizer.zero_grad()
        preds = model(x_cont, x_cat)
        if task_type == 'regression':
            preds = preds.flatten()
        loss = loss_fn(preds, y)
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
    return sum(losses) / len(losses)


@torch.no_grad()
def evaluate(model, dataloader, device, metric_func, task_type):
    model.eval()
    preds_all = []
    y_all = []
    for batch in dataloader:
        if len(batch) == 3:
            x_cont, x_cat, y = batch
            x_cont, x_cat, y = x_cont.to(device), x_cat.to(device), y.to(device)
        if len(batch) == 2:
            x_cont, y = batch
            x_cont, y = x_cont.to(device), y.to(device)
            x_cat = None
        preds = model(x_cont, x_cat)
        if task_type == 'regression':
            preds = preds.flatten()
        preds_all.append(preds.cpu().detach())
        y_all.append(y.cpu().detach())
    preds_all = torch.cat(preds_all)
    if task_type == 'classification':
        preds_all = preds_all.argmax(dim=1)
    preds_all = preds_all.numpy()
    y_all = torch.cat(y_all).numpy()
    return metric_func(y_all, preds_all)


def run_experiment(model, optimizer, dataloaders, loss_fn, device, metric_func, 
                   task_type, n_epoches, maximize=True, y_std=None):
    model.to(device)
    best_val = None
    best_test = None
    best_step = None
    for i in (pbar := tqdm(range(1, n_epoches + 1))):
        loss = train_step(model, optimizer, dataloaders['train'], loss_fn, device, task_type)
        metric_val = evaluate(model, dataloaders['val'], device, metric_func, task_type)
        metric_test = evaluate(model, dataloaders['test'], device, metric_func, task_type)
        str_desc = f'Loss: {loss}, val metric {metric_val}, test metric {metric_test}'
        pbar.set_description(str_desc)
        if best_val is None or (maximize and metric_val > best_val) or (not maximize and metric_val < best_val):
            best_val = metric_val
            best_test = metric_test
            best_step = i
    if task_type == 'regression' and y_std:
        best_val *= y_std
        best_test *= y_std
    return {
        'model': model.cpu(),
        'val': best_val,
        'test': best_test,
        'best_step': best_step
    }

def experiments_series(experiment_name, model_class, model_args, model_kwargs,
                       optimizer_class, learning_rate,
                       dataloaders, loss_fn, device, metric_func, 
                       task_type, n_epoches, maximize, n_runs):
    print(f'===== Running experiment "{experiment_name}" =====')
    val_results = []
    test_results = []
    for _ in tqdm(range(n_runs)):
        model = model_class(*model_args, **model_kwargs)
        optimizer = optimizer_class(model.parameters(), lr=learning_rate)
        results = run_experiment(model, optimizer, dataloaders, loss_fn, device, metric_func,
                                task_type, n_epoches, maximize)
        val_results.append(results['val'])
        test_results.append(results['test'])
    val_results = np.array(val_results)
    test_results = np.array(test_results)
    
    print('===== Experiments results =====')
    print(f'Validation metric: {val_results.mean()}±{val_results.std()}')
    print(f'Corresponding test metric: {test_results.mean()}±{test_results.std()}')

#### Model

In [6]:
class BasicTabDLModel(nn.Module):
    def __init__(self, n_cont_features, cat_cardinalities, mlp_kwargs, model_type):
        super().__init__()
        self.cat_cardinalities = cat_cardinalities
        d_cat = sum(cat_cardinalities)

        
        if model_type == 'MLP-PLR':
            d_embedding = 24
            self.cont_embeddings = layers.PeriodicEmbeddings(n_cont_features, d_embedding, lite=False)
            d_num = n_cont_features * d_embedding
        elif model_type == 'MLP':
            self.cont_embeddings = nn.Identity()
            d_num = n_cont_features
        
        self.backbone = layers.MLP(d_in=d_num + d_cat, **mlp_kwargs)

    def forward(self, x_cont, x_cat):
        x = []
        x.append(self.cont_embeddings(x_cont).flatten(1))
        if x_cat is not None:
            x.extend(
                F.one_hot(column, cardinality)
                for column, cardinality in zip(x_cat.T, self.cat_cardinalities)
            )
        x = torch.column_stack(x)
        x = self.backbone(x)
        return x

#### Experiment: regression on California Housing dataset

There is no categorical features. Let's prepare data. First of all we will read it

In [7]:
dataset = sklearn.datasets.fetch_california_housing()
X_cont = dataset["data"]
Y = dataset["target"]
X_cont = X_cont.astype(np.float32)
n_cont_features = X_cont.shape[1]
Y = Y.astype(np.float32)

Now let's make train/val/test split

In [8]:
all_idx = np.arange(len(Y))
trainval_idx, test_idx = sklearn.model_selection.train_test_split(
    all_idx, train_size=0.8, random_state=42
)
train_idx, val_idx = sklearn.model_selection.train_test_split(
    trainval_idx, train_size=0.8, random_state=42
)

X_train, y_train = X_cont[train_idx], Y[train_idx]
X_val, y_val = X_cont[val_idx], Y[val_idx]
X_test, y_test = X_cont[test_idx], Y[test_idx]

We will use quantile tranform with noise to obtain initial embeddings for numerical features

In [9]:
noise = np.random.default_rng(42).normal(0.0, 1e-5, X_train.shape).astype(X_train.dtype)
preprocessing = sklearn.preprocessing.QuantileTransformer(
    n_quantiles=max(min(len(train_idx) // 30, 1000), 10),
    output_distribution="normal",
    subsample=10**9,
)
preprocessing.fit(X_train + noise)

QuantileTransformer(n_quantiles=440, output_distribution='normal',
                    subsample=1000000000)

In [10]:
X_train = preprocessing.transform(X_train)
X_val = preprocessing.transform(X_val)
X_test = preprocessing.transform(X_test)

Also let's normalize target, it will make training process more stable

In [11]:
y_mean, y_std = y_train.mean(), y_train.std()
y_train = (y_train - y_mean) / y_std
y_val = (y_val - y_mean) / y_std
y_test = (y_test - y_mean) / y_std

And now we can make dataloaders

In [12]:
dataset_train = data.TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
dataloader_train = data.DataLoader(
    dataset=dataset_train,
    batch_size=batch_size,
    num_workers=num_workers,
    shuffle=True,
)

dataset_val = data.TensorDataset(torch.tensor(X_val), torch.tensor(y_val))
dataloader_val = data.DataLoader(
    dataset=dataset_val,
    batch_size=batch_size,
    num_workers=num_workers,
    shuffle=True,
)

dataset_test = data.TensorDataset(torch.tensor(X_test), torch.tensor(y_test))
dataloader_test = data.DataLoader(
    dataset=dataset_test,
    batch_size=batch_size,
    num_workers=num_workers,
    shuffle=True,
)

In [13]:
dataloaders = {
    'train': dataloader_train,
    'val': dataloader_val,
    'test': dataloader_test
}

That's all! Let's run experiments

In [14]:
experiments_series(
    experiment_name='Simple MLP on California Housing data',
    model_class=BasicTabDLModel,
    model_args=[],
    model_kwargs={
        'n_cont_features': n_cont_features,
        'cat_cardinalities': [], 
        'mlp_kwargs':     {
                            "d_layers": [384, 384],
                            "dropouts": [0.4, 0.4],
                            "activation": nn.ReLU,
                            "d_out": 1,
                          },
        'model_type': 'MLP'
    },
    optimizer_class=torch.optim.Adam,
    learning_rate=3e-4,
    dataloaders=dataloaders,
    loss_fn=F.mse_loss,
    device=device,
    metric_func=functools.partial(sklearn.metrics.mean_squared_error, squared=False),
    task_type='regression',
    n_epoches=100,
    maximize=False,
    n_runs=10
)

===== Running experiment "Simple MLP on California Housing data" =====


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

===== Experiments results =====
Validation metric: 0.4749775528907776±0.0013345532352104783
Corresponding test metric: 0.46647271513938904±0.0023757850285619497


In [15]:
experiments_series(
    experiment_name='MLP-PLR on California Housing data',
    model_class=BasicTabDLModel,
    model_args=[],
    model_kwargs={
        'n_cont_features': n_cont_features,
        'cat_cardinalities': [], 
        'mlp_kwargs':     {
                            "d_layers": [384, 384],
                            "dropouts": [0.4, 0.4],
                            "activation": nn.ReLU,
                            "d_out": 1,
                          },
        'model_type': 'MLP-PLR'
    },
    optimizer_class=torch.optim.Adam,
    learning_rate=3e-4,
    dataloaders=dataloaders,
    loss_fn=F.mse_loss,
    device=device,
    metric_func=functools.partial(sklearn.metrics.mean_squared_error, squared=False),
    task_type='regression',
    n_epoches=100,
    maximize=False,
    n_runs=10
)

===== Running experiment "MLP-PLR on California Housing data" =====


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

===== Experiments results =====
Validation metric: 0.43340855836868286±0.0029671844094991684
Corresponding test metric: 0.4137554168701172±0.0032218550331890583


#### Experiment: multi-class classification on Covertype dataset

First 10 columns are numerical, other are categorical (binary). Also we want to shift targets by one so that they start from 0

In [16]:
dataset = sklearn.datasets.fetch_covtype()

In [17]:
X_cont = dataset["data"][:, :10]
X_cat = dataset["data"][:, 10:]
Y = dataset["target"]
X_cont = X_cont.astype(np.float32)
X_cat = X_cat.astype(np.int64)
n_cont_features = X_cont.shape[1]
Y = Y.astype(np.int64) - 1

Let's make split again

In [18]:
all_idx = np.arange(len(Y))
trainval_idx, test_idx = sklearn.model_selection.train_test_split(
    all_idx, train_size=0.8, random_state=42
)
train_idx, val_idx = sklearn.model_selection.train_test_split(
    trainval_idx, train_size=0.8, random_state=42
)

X_cont_train, X_cat_train, y_train = X_cont[train_idx], X_cat[train_idx], Y[train_idx]
X_cont_val, X_cat_val, y_val = X_cont[val_idx], X_cat[val_idx], Y[val_idx]
X_cont_test, X_cat_test, y_test = X_cont[test_idx], X_cat[test_idx], Y[test_idx]

The same preprocessing as in California Housing

In [19]:
noise = np.random.default_rng(42).normal(0.0, 1e-5, X_cont_train.shape).astype(X_cont_train.dtype)
preprocessing = sklearn.preprocessing.QuantileTransformer(
    n_quantiles=max(min(len(train_idx) // 30, 1000), 10),
    output_distribution="normal",
    subsample=10**9,
)
preprocessing.fit(X_cont_train + noise)

QuantileTransformer(output_distribution='normal', subsample=1000000000)

In [20]:
X_cont_train = preprocessing.transform(X_cont_train)
X_cont_val = preprocessing.transform(X_cont_val)
X_cont_test = preprocessing.transform(X_cont_test)

Dataloaders:

In [21]:
dataset_train = data.TensorDataset(torch.tensor(X_cont_train), torch.tensor(X_cat_train), torch.tensor(y_train))
dataloader_train = data.DataLoader(
    dataset=dataset_train,
    batch_size=batch_size,
    num_workers=num_workers,
    shuffle=True,
)

dataset_val = data.TensorDataset(torch.tensor(X_cont_val), torch.tensor(X_cat_val), torch.tensor(y_val))
dataloader_val = data.DataLoader(
    dataset=dataset_val,
    batch_size=batch_size,
    num_workers=num_workers,
    shuffle=True,
)

dataset_test = data.TensorDataset(torch.tensor(X_cont_test), torch.tensor(X_cat_test), torch.tensor(y_test))
dataloader_test = data.DataLoader(
    dataset=dataset_test,
    batch_size=batch_size,
    num_workers=num_workers,
    shuffle=True,
)

In [22]:
dataloaders = {
    'train': dataloader_train,
    'val': dataloader_val,
    'test': dataloader_test
}

And experiments!

In [24]:
experiments_series(
    experiment_name='Simple MLP on Covertype data',
    model_class=BasicTabDLModel,
    model_args=[],
    model_kwargs={
        'n_cont_features': n_cont_features,
        'cat_cardinalities': 2 * np.ones(shape=X_cat_train.shape[1], dtype=int), 
        'mlp_kwargs':     {
                            "d_layers": [384, 384],
                            "dropouts": [0.4, 0.4],
                            "activation": nn.ReLU,
                            "d_out": 7,
                          },
        'model_type': 'MLP'
    },
    optimizer_class=torch.optim.Adam,
    learning_rate=3e-4,
    dataloaders=dataloaders,
    loss_fn=F.cross_entropy,
    device=device,
    metric_func=sklearn.metrics.accuracy_score,
    task_type='classification',
    n_epoches=100,
    maximize=True,
    n_runs=5
)

===== Running experiment "Simple MLP on Covertype data" =====


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

===== Experiments results =====
Validation metric: 0.9115466534713109±0.0007867657656984384
Corresponding test metric: 0.9111847370549814±0.0011736211405357995


In [25]:
experiments_series(
    experiment_name='MLP-PLR on Covertype data',
    model_class=BasicTabDLModel,
    model_args=[],
    model_kwargs={
        'n_cont_features': n_cont_features,
        'cat_cardinalities': 2 * np.ones(shape=X_cat_train.shape[1], dtype=int), 
        'mlp_kwargs':     {
                            "d_layers": [384, 384],
                            "dropouts": [0.4, 0.4],
                            "activation": nn.ReLU,
                            "d_out": 7,
                          },
        'model_type': 'MLP-PLR'
    },
    optimizer_class=torch.optim.Adam,
    learning_rate=3e-4,
    dataloaders=dataloaders,
    loss_fn=F.cross_entropy,
    device=device,
    metric_func=sklearn.metrics.accuracy_score,
    task_type='classification',
    n_epoches=100,
    maximize=True,
    n_runs=5
)

===== Running experiment "MLP-PLR on Covertype data" =====


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

===== Experiments results =====
Validation metric: 0.9458208730449±0.0004978380121563677
Corresponding test metric: 0.9448688932299512±0.0006910401107723556
